# PyCitySchools

### Adding the Pandas dependency

In [13]:
import pandas as pd
import os

school_data_to_load = os.path.join("..","Resources", "schools_complete.csv")
student_data_to_load = os.path.join("..","Resources", "students_complete.csv")

school_data_df = pd.read_csv(school_data_to_load)
student_data_df = pd.read_csv(student_data_to_load)

### Add each prefix and suffix to remove to a list. Iterate through the words in the "prefixes_suffixes" list and replace them with an empty space, "".

In [14]:
prefixes_suffixes = ["Dr. ", "Mr. ","Ms. ", "Mrs. ", "Miss ", " MD", " DDS", " DVM", " PhD"]

for word in prefixes_suffixes:
    student_data_df["student_name"] = student_data_df["student_name"].str.replace(word,"")

### Counting and declaring variables

In [15]:
school_data_complete_df = pd.merge(student_data_df, school_data_df, on=["school_name", "school_name"])

student_count = school_data_complete_df["Student ID"].count()
school_count = len(school_data_complete_df["school_name"].unique())
total_budget = school_data_complete_df["budget"].unique().sum()

### Creating function for calculating percentages

In [16]:
def percentage_calc(count, total_count):
    return count/float(total_count)* 100

### Determining Average and count student passing grades

In [17]:
average_reading_score = school_data_complete_df["reading_score"].mean()
average_math_score = school_data_complete_df["math_score"].mean()

passing_reading = school_data_complete_df[school_data_complete_df["reading_score"] >= 70]
passing_math = school_data_complete_df[school_data_complete_df["math_score"] >= 70]

passing_reading_count = passing_reading["student_name"].count()
passing_math_count = passing_math["student_name"].count()

passing_reading_percentage = percentage_calc(passing_reading_count, student_count)
passing_math_percentage = percentage_calc(passing_math_count, student_count)

passing_both = school_data_complete_df[(school_data_complete_df["reading_score"] >= 70) & (school_data_complete_df["math_score"] >= 70)]
passing_both_count = passing_both["student_name"].count()
passing_both_percentage = percentage_calc(passing_both_count, student_count)

### District Summary DataFrame / Changing formate for Budget, Average Scores, and Score Percentages

In [18]:
district_summary_df = pd.DataFrame([{"Total Schools": school_count, "Total Students": student_count, "Total Budget": total_budget, "Average Reading Score": average_reading_score, "Average Math Score": average_math_score, "% Passing Reading": passing_reading_percentage, "% Passing Math": passing_math_percentage, "% Overall Passing": passing_both_percentage}])

district_summary_df["Total Students"] = district_summary_df["Total Students"].map("{:,}".format)
district_summary_df["Total Budget"] = district_summary_df["Total Budget"].map("${:,.2f}".format)
district_summary_df["Average Reading Score"] = district_summary_df["Average Reading Score"].map("{:.1f}".format)
district_summary_df["Average Math Score"] = district_summary_df["Average Math Score"].map("{:.1f}".format)
district_summary_df["% Passing Reading"] = district_summary_df["% Passing Reading"].map("{:.2f}%".format)
district_summary_df["% Passing Math"] = district_summary_df["% Passing Math"].map("{:.2f}%".format)
district_summary_df["% Overall Passing"] = district_summary_df["% Overall Passing"].map("{:.2f}%".format)

new_column_order = ["Total Schools", "Total Students", "Total Budget","Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing"]
district_summary_df = district_summary_df[new_column_order]
district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",79.0,81.9,74.98%,85.81%,65.17%


### Getting school type, count, budget, 

In [19]:
per_school_types = school_data_df.set_index(["school_name"])["type"]
per_school_budget = school_data_df.set_index(["school_name"])["budget"]
per_school_counts = school_data_complete_df["school_name"].value_counts()
per_school_averages = school_data_complete_df.groupby(["school_name"]).mean()

per_school_math_average = school_data_complete_df.groupby(["school_name"]).mean()["math_score"]
per_school_reading_average = school_data_complete_df.groupby(["school_name"]).mean()["reading_score"]

per_school_capita = per_school_budget / per_school_counts
per_school_types_df = pd.DataFrame(per_school_types)

### Calculate the passing scores by creating a filtered DataFrame.

In [20]:
per_school_passing_math = school_data_complete_df[(school_data_complete_df["math_score"] >= 70)]
per_school_passing_reading = school_data_complete_df[(school_data_complete_df["reading_score"] >= 70)]

per_school_passing_math_count = per_school_passing_math.groupby(["school_name"]).count()["student_name"]
per_school_passing_reading_count = per_school_passing_reading.groupby(["school_name"]).count()["student_name"]

per_school_passing_math_per = per_school_passing_math_count / per_school_counts * 100
per_school_passing_reading_per = per_school_passing_reading_count / per_school_counts* 100

In [21]:
per_passing_math_reading = school_data_complete_df[(school_data_complete_df["math_score"] >= 70) & (school_data_complete_df["reading_score"] >= 70)]
per_passing_math_reading_count = per_passing_math_reading.groupby(["school_name"]).count()["student_name"]
per_overall_passing_percentage = per_passing_math_reading_count / per_school_counts * 100

In [22]:
per_school_summary_df = pd.DataFrame({
             "School Type": per_school_types,
             "Total Students": per_school_counts,
             "Total School Budget": per_school_budget,
             "Per Student Budget": per_school_capita,
             "Average Math Score": per_school_math_average,
           "Average Reading Score": per_school_reading_average,
           "% Passing Math": per_school_passing_math_per,
           "% Passing Reading": per_school_passing_reading_per,
           "% Overall Passing": per_overall_passing_percentage})

# per_school_summary_df["Total School Budget"] = per_school_summary_df["Total School Budget"].map("${:,.2f}".format)
# per_school_summary_df["Per Student Budget"] = per_school_summary_df["Per Student Budget"].map("${:,.2f}".format)
# per_school_summary_df["Average Math Score"] = per_school_summary_df["Average Math Score"].map("{:,.1f}%".format)
# per_school_summary_df["Average Reading Score"] = per_school_summary_df["Average Reading Score"].map("{:,.1f}%".format)
# per_school_summary_df["% Passing Math"] = per_school_summary_df["% Passing Math"].map("{:,.1f}%".format)
# per_school_summary_df["% Passing Reading"] = per_school_summary_df["% Passing Reading"].map("{:,.1f}%".format)
# per_school_summary_df["% Overall Passing"] = per_school_summary_df["% Overall Passing"].map("{:,.1f}%".format)

# Reorder the columns in the order you want them to appear.
new_column_order = ["School Type", "Total Students", "Total School Budget", "Per Student Budget", "Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing"]
per_school_summary_df = per_school_summary_df[new_column_order]
top_schools = per_school_summary_df.sort_values(["% Overall Passing"], ascending=False)
# Assign district summary df the new column order.

top_schools.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,1043130,638.0,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,585858,609.0,83.839917,84.044699,94.594595,95.945946,90.540541


### Creating Grade_level Dataframes

In [23]:
ninth_graders = school_data_complete_df[(school_data_complete_df["grade"] == "9th")]
tenth_graders = school_data_complete_df[(school_data_complete_df["grade"] == "10th")]
eleventh_graders = school_data_complete_df[(school_data_complete_df["grade"] == "11th")]
twelfth_graders = school_data_complete_df[(school_data_complete_df["grade"] == "12th")]

ninth_grader_math_scores = ninth_graders.groupby(["school_name"]).mean()["math_score"]
tenth_grader_math_scores = tenth_graders.groupby(["school_name"]).mean()["math_score"]
eleventh_grader_math_scores = eleventh_graders.groupby(["school_name"]).mean()["math_score"]
twelfth_grader_math_scores = twelfth_graders.groupby(["school_name"]).mean()["math_score"]

ninth_grader_reading_scores = ninth_graders.groupby(["school_name"]).mean()["reading_score"]
tenth_grader_reading_scores = tenth_graders.groupby(["school_name"]).mean()["reading_score"]
eleventh_grader_reading_scores = eleventh_graders.groupby(["school_name"]).mean()["reading_score"]
twelfth_grader_reading_scores = twelfth_graders.groupby(["school_name"]).mean()["reading_score"]

math_scores_by_grade = pd.DataFrame({
               "9th": ninth_grader_math_scores,
               "10th": tenth_grader_math_scores,
               "11th": eleventh_grader_math_scores,
               "12th": twelfth_grader_math_scores})

reading_scores_by_grade = pd.DataFrame({
               "9th": ninth_grader_reading_scores,
               "10th": tenth_grader_reading_scores,
               "11th": eleventh_grader_reading_scores,
               "12th": twelfth_grader_reading_scores})

# Format each grade column.
math_scores_by_grade["9th"] = math_scores_by_grade["9th"].map("{:.1f}".format)
math_scores_by_grade["10th"] = math_scores_by_grade["10th"].map("{:.1f}".format)
math_scores_by_grade["11th"] = math_scores_by_grade["11th"].map("{:.1f}".format)
math_scores_by_grade["12th"] = math_scores_by_grade["12th"].map("{:.1f}".format)

reading_scores_by_grade["9th"] = reading_scores_by_grade["9th"].map("{:,.1f}".format)
reading_scores_by_grade["10th"] = reading_scores_by_grade["10th"].map("{:,.1f}".format)
reading_scores_by_grade["11th"] = reading_scores_by_grade["11th"].map("{:,.1f}".format)
reading_scores_by_grade["12th"] = reading_scores_by_grade["12th"].map("{:,.1f}".format)

  # Remove the index name.
math_scores_by_grade.index.name = None
reading_scores_by_grade.index.name = None

### Establishing the Spending Ranges

In [24]:
per_school_capita.describe()
spending_bins = [0, 585, 630, 645, 675]
group_names = ["<$584", "$585-629", "$630-644", "$645-675"]
per_school_capita.groupby(pd.cut(per_school_capita, spending_bins)).count()
per_school_summary_df["Spending Ranges (Per Student)"] = pd.cut(per_school_capita, spending_bins, labels=group_names)

per_school_summary_df

# spending_math_scores = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Math Score"]

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing,Spending Ranges (Per Student)
Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,66.680064,81.933280,54.642283,$585-629
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,91.334769,<$584
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,53.204476,$630-644
Ford High School,District,2739,1763916,644.0,77.102592,80.746258,68.309602,79.299014,54.289887,$630-644
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,90.599455,$585-629
Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,66.752967,80.862999,53.527508,$645-675
Holden High School,Charter,427,248087,581.0,83.803279,83.814988,92.505855,96.252927,89.227166,<$584
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,53.513884,$645-675
Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,66.057551,81.222432,53.539172,$645-675
Pena High School,Charter,962,585858,609.0,83.839917,84.044699,94.594595,95.945946,90.540541,$585-629


### Grouping by Spending Ranges

In [25]:
spending_math_scores = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Math Score"]
spending_reading_scores = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Reading Score"]
spending_passing_math = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Math"]
spending_passing_reading = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Reading"]
overall_passing_spending = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Overall Passing"]

spending_summary_df = pd.DataFrame({
    "Average Math Score" : spending_math_scores,
    "Average Reading Score" : spending_reading_scores,
    "% Passing Math": spending_passing_math,
    "% Passing Reading": spending_passing_reading,
    "% Overall Passing": overall_passing_spending})

# Formatting
spending_summary_df["Average Math Score"] = spending_summary_df["Average Math Score"].map("{:.1f}".format)
spending_summary_df["Average Reading Score"] = spending_summary_df["Average Reading Score"].map("{:.1f}".format)
spending_summary_df["% Passing Math"] = spending_summary_df["% Passing Math"].map("{:.0f}".format)
spending_summary_df["% Passing Reading"] = spending_summary_df["% Passing Reading"].map("{:.0f}".format)
spending_summary_df["% Overall Passing"] = spending_summary_df["% Overall Passing"].map("{:.0f}".format)
spending_summary_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$584,83.5,83.9,93,97,90
$585-629,81.9,83.2,87,93,81
$630-644,78.5,81.6,73,84,63
$645-675,77.0,81.0,66,81,54


### Creating Bins Per School

In [28]:
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]
per_school_summary_df["School Size"] = pd.cut(per_school_summary_df["Total Students"], size_bins, labels=group_names)

size_math_scores = per_school_summary_df.groupby(["School Size"]).mean()["Average Math Score"]
size_reading_scores = per_school_summary_df.groupby(["School Size"]).mean()["Average Reading Score"]
size_passing_math = per_school_summary_df.groupby(["School Size"]).mean()["% Passing Math"]
size_passing_reading = per_school_summary_df.groupby(["School Size"]).mean()["% Passing Reading"]
size_overall_passing = per_school_summary_df.groupby(["School Size"]).mean()["% Overall Passing"]

size_summary_df = pd.DataFrame({
          "Average Math Score" : size_math_scores,
          "Average Reading Score": size_reading_scores,
          "% Passing Math": size_passing_math,
          "% Passing Reading": size_passing_reading,
          "% Overall Passing": size_overall_passing})

size_summary_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,89.883853
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,90.621535
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,58.286003


### Grouping by School Type

In [29]:
type_math_scores = per_school_summary_df.groupby(["School Type"]).mean()["Average Math Score"]
type_reading_scores = per_school_summary_df.groupby(["School Type"]).mean()["Average Reading Score"]
type_passing_math = per_school_summary_df.groupby(["School Type"]).mean()["% Passing Math"]
type_passing_reading = per_school_summary_df.groupby(["School Type"]).mean()["% Passing Reading"]
type_overall_passing = per_school_summary_df.groupby(["School Type"]).mean()["% Overall Passing"]

type_summary_df = pd.DataFrame({
          "Average Math Score" : type_math_scores,
          "Average Reading Score": type_reading_scores,
          "% Passing Math": type_passing_math,
          "% Passing Reading": type_passing_reading,
          "% Overall Passing": type_overall_passing})

type_summary_df["Average Math Score"] = type_summary_df["Average Math Score"].map("{:.1f}".format)

type_summary_df["Average Reading Score"] = type_summary_df["Average Reading Score"].map("{:.1f}".format)

type_summary_df["% Passing Math"] = type_summary_df["% Passing Math"].map("{:.0f}".format)

type_summary_df["% Passing Reading"] = type_summary_df["% Passing Reading"].map("{:.0f}".format)

type_summary_df["% Overall Passing"] = type_summary_df["% Overall Passing"].map("{:.0f}".format)

type_summary_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.5,83.9,94,97,90
District,77.0,81.0,67,81,54
